In [1]:
import psycopg2
import pandas as pd

In [34]:
# https://docs.flashbots.net/flashbots-data/mev-inspect-py/overview

In [12]:
param_dic = {
    "host"      : "localhost",
    "database"  : "mev_inspect",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

conn = psycopg2.connect(
    database=param_dic['database'], 
    user=param_dic['user'], 
    password=param_dic['password'], 
    host=param_dic['host'], 
    port=param_dic['port']
)

In [30]:
cur = conn.cursor()
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
table_names = cur.fetchall()
table_names = [name_tuple[0] for name_tuple in table_names]
cur.close()

In [21]:
blocks = pd.read_sql_query('select * from blocks',con=conn)

In [22]:
blocks

,block_number,block_timestamp
0,12914947.0,2021-07-28 14:29:44
1,12914949.0,2021-07-28 14:30:17
2,12914944.0,2021-07-28 14:28:24
3,12914946.0,2021-07-28 14:29:36
4,12914945.0,2021-07-28 14:29:30
...,...,...
5974,13781053.0,2021-12-11 02:06:38
5975,13781047.0,2021-12-11 02:05:30
5976,13781052.0,2021-12-11 02:06:21
5977,13781055.0,2021-12-11 02:07:10


In [25]:
blocks[(blocks['block_timestamp'] > '2021-12-09') & (blocks['block_timestamp'] < '2021-12-11')]

,block_number,block_timestamp
10,13775086.0,2021-12-10 03:56:34
11,13775084.0,2021-12-10 03:56:11
12,13775087.0,2021-12-10 03:56:50
13,13775083.0,2021-12-10 03:55:43
14,13775085.0,2021-12-10 03:56:15
...,...,...
5419,13780496.0,2021-12-10 23:57:45
5420,13780497.0,2021-12-10 23:57:59
5421,13780499.0,2021-12-10 23:58:43
5422,13780500.0,2021-12-10 23:59:32


In [26]:
table_names

['transfers',
 'blocks',
 'alembic_version',
 'classified_traces',
 'arbitrages',
 'arbitrage_swaps',
 'swaps',
 'miner_payments',
 'latest_block_update',
 'liquidations',
 'punk_snipes',
 'punk_bids',
 'punk_bid_acceptances',
 'prices']

In [31]:
table_names.remove('alembic_version')
table_names.remove('classified_traces')

In [33]:
for table_name in table_names:
    df = pd.read_sql_query('select * from {}'.format(table_name), con=conn)
    df.to_csv('../../data/mev_inspect/{}.csv'.format(table_name), index=False)

In [11]:
conn.close()